##### **Parte 1. Análisis Descriptivo del Dataset**


In [33]:
# Se realizan las importaciones necesarias.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import seaborn as sns


In [34]:
# Se carga el dataframe.
df = pd.read_csv('avocado.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


In [35]:
"""
Tenemos bastante información, pero nos decidimos por quedarnos con las siguientes columnas:
date, average_price, total volume, total_bags, small_bags, large_bags, xlarge_bags, type, year, region.
"""
#Cambiamos nombres de columnas. Unnamed: 0 a index, AveragePrice por AvgPrice.
df.rename(columns={'AveragePrice': 'AvgPrice'}, inplace=True)

# Droppeamos columnas innecesarias.
df.drop(columns=['Unnamed: 0', '4046', '4225', '4770'], inplace=True)
df.head()


,Date,AvgPrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [ ]:
# Date cambiara de tipo object a datetime64[ns].
df['Date'] = pd.to_datetime(df['Date'])

# No necesitamos el año, ya que lo tenemos en la columna year, por lo que Date no mostrara el año.
df['Date'] = df['Date'].dt.strftime('%m-%d')



In [38]:
# Cambiamos las columnas de tipo object a category.
df['type'] = df['type'].astype('category')
df['region'] = df['region'].astype('category')

In [ ]:
#Ahora, vamos a generar visualizaciones exploratorias con Matplotlib y Seaborn.
# Histogramas de las variables numéricas
df.hist(figsize=(10, 10), bins=30)
plt.suptitle('Histograms of Numerical Variables')
plt.show()

# Boxplots para detectar outliers
df.boxplot(figsize=(10, 10), rot=90)
plt.suptitle('Boxplots of Numerical Variables')
plt.show()

# añadir mas despues XDD

